In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import scipy
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from imblearn.under_sampling import ClusterCentroids
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from imblearn.over_sampling import ADASYN,SMOTE,BorderlineSMOTE
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score,recall_score,hamming_loss
from sklearn.feature_selection import SelectPercentile,chi2

In [ ]:
data=scipy.io.loadmat('./drive/MyDrive/Colab Notebooks/mydata/PSTAAP.mat')#读入由matlab处理好的样本特征向量
data_test=scipy.io.loadmat('./drive/MyDrive/Colab Notebooks/mydata/PSTAAP_test.mat')

In [ ]:
data['PSTAAP1'][0].shape#每个样本94个特征

(46,)

In [ ]:
data['PSTAAP1']

array([[-2.49204046e-04, -5.18771351e-05, -1.37672739e-04, ...,
         7.27248879e-05,  0.00000000e+00, -6.23010115e-05],
       [-6.23010115e-05, -1.86903034e-04,  1.10768172e-04, ...,
         5.89772503e-05, -2.68183532e-05,  3.84832940e-05],
       [-1.24602023e-04, -1.24602023e-04, -2.49204046e-04, ...,
         0.00000000e+00, -1.37476376e-05, -3.32376113e-06],
       ...,
       [-2.62951683e-04, -4.14532586e-05, -6.23010115e-05, ...,
        -2.34945921e-05, -6.23010115e-05,  0.00000000e+00],
       [-1.48419740e-04, -1.24602023e-04, -7.60486490e-05, ...,
        -6.23010115e-05,  0.00000000e+00, -8.62133229e-08],
       [-8.57956036e-05,  2.96228592e-03, -1.86903034e-04, ...,
        -6.23872248e-05, -9.88296452e-08,  3.78264276e-04]])

In [ ]:
data['PSTAAP1'].shape#第一类样本数量为9279

(9279, 46)

### 先将数据读入到矩阵中便于处理

In [ ]:
d_1=np.array(data['PSTAAP1'])
d_2=np.array(data['PSTAAP2'])
d_3=np.array(data['PSTAAP3'])
d_4=np.array(data['PSTAAP4'])
d_5=np.array(data['PSTAAP5'])
d_6=np.array(data['PSTAAP6'])
d_7=np.array(data['PSTAAP7'])
d_8=np.array(data['PSTAAP8'])
d_9=np.array(data['PSTAAP9'])
d_10=np.array(data['PSTAAP10'])
d_11=np.array(data['PSTAAP11'])

dt_1=np.array(data_test['PSTAAP1'])
dt_2=np.array(data_test['PSTAAP2'])
dt_3=np.array(data_test['PSTAAP3'])
dt_4=np.array(data_test['PSTAAP4'])
dt_5=np.array(data_test['PSTAAP5'])
dt_6=np.array(data_test['PSTAAP6'])
dt_7=np.array(data_test['PSTAAP7'])
dt_8=np.array(data_test['PSTAAP8'])
dt_9=np.array(data_test['PSTAAP9'])
dt_10=np.array(data_test['PSTAAP10'])
dt_11=np.array(data_test['PSTAAP11'])


### 接下来进行数据平衡

#### 整合数据

In [ ]:
X_train=np.concatenate((d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11),axis=0)
X_test=np.concatenate((dt_1,dt_2,dt_3,dt_4,dt_5,dt_6,dt_7,dt_8,dt_9,dt_10,dt_11),axis=0)
print(X_train.shape)
print(X_test.shape)

(12983, 46)
(5662, 46)


In [ ]:
for i in range(1,12):
    exec(f'y_{i}=np.full(d_{i}.shape[0],i)');
    exec(f'yt_{i}=np.full(dt_{i}.shape[0],i)')
y_dir=[y_1,y_2,y_3,y_4,y_5,y_6,y_7,y_8,y_9,y_10,y_11]
yt_dir=[yt_1,yt_2,yt_3,yt_4,yt_5,yt_6,yt_7,yt_8,yt_9,yt_10,yt_11]
y=np.concatenate((y_dir),axis=0)
yt=np.concatenate((yt_dir),axis=0)
print(y.shape)
print(yt.shape)

(12983,)
(5662,)


In [ ]:
def ratio_multiplier(y):
    from collections import Counter

    multiplier = {1: 0.334}
    target_stats = Counter(y)
    for key, value in target_stats.items():
        if key in multiplier:
            target_stats[key] = int(value * multiplier[key])
    return target_stats

ratio_multiplier(y)

Counter({1: 3099,
         2: 710,
         3: 600,
         4: 454,
         5: 561,
         6: 251,
         7: 360,
         8: 88,
         9: 153,
         10: 454,
         11: 73})

In [ ]:
under=ClusterCentroids(
    sampling_strategy=ratio_multiplier,random_state=1,
    estimator=MiniBatchKMeans(n_init=1,random_state=1)
)
x_under_sampled,y_under_sampled=under.fit_resample(X_train,y)
over1=BorderlineSMOTE(random_state=1)
over2=SMOTE(random_state=1)
x_resampled1,y_resampled1=over1.fit_resample(x_under_sampled,y_under_sampled)
x_resampled,y_resampled=over2.fit_resample(x_resampled1,y_resampled1)
for i in range(1,12):
    print(sum(y_resampled==i))

3099
3099
3099
3099
3099
3099
3099
3099
3099
3099
3099


In [ ]:
X_train=x_resampled
Y_train=np.zeros((y_resampled.shape[0],4))
Y_test=np.zeros((yt.shape[0],4))
for i in range(y_resampled.shape[0]):
    if y_resampled[i]==1:
        Y_train[i]=np.array([1,0,0,0])#a
    elif y_resampled[i]==2:
        Y_train[i]=np.array([0,1,0,0])#c
    elif y_resampled[i]==3:
        Y_train[i]=np.array([0,0,1,0])#m
    elif y_resampled[i]==4:
        Y_train[i]=np.array([0,0,0,1])#s
    elif y_resampled[i]==5:
        Y_train[i]=np.array([1,1,0,0])#ac
    elif y_resampled[i]==6:
        Y_train[i]=np.array([1,0,1,0])#am
    elif y_resampled[i]==7:
        Y_train[i]=np.array([1,0,0,1])#as
    elif y_resampled[i]==8:
        Y_train[i]=np.array([0,1,1,0])#cm
    elif y_resampled[i]==9:
        Y_train[i]=np.array([1,1,1,0])#acm
    elif y_resampled[i]==10:
        Y_train[i]=np.array([1,1,0,1])#acs
    elif y_resampled[i]==11:
        Y_train[i]=np.array([1,1,1,1])#acms

for i in range(yt.shape[0]):
    if yt[i]==1:
        Y_test[i]=np.array([1,0,0,0])#a
    elif yt[i]==2:
        Y_test[i]=np.array([0,1,0,0])#c
    elif yt[i]==3:
        Y_test[i]=np.array([0,0,1,0])#m
    elif yt[i]==4:
        Y_test[i]=np.array([0,0,0,1])#s
    elif yt[i]==5:
        Y_test[i]=np.array([1,1,0,0])#ac
    elif yt[i]==6:
        Y_test[i]=np.array([1,0,1,0])#am
    elif yt[i]==7:
        Y_test[i]=np.array([1,0,0,1])#as
    elif yt[i]==8:
        Y_test[i]=np.array([0,1,1,0])#cm
    elif yt[i]==9:
        Y_test[i]=np.array([1,1,1,0])#acm
    elif yt[i]==10:
        Y_test[i]=np.array([1,1,0,1])#acs
    elif yt[i]==11:
        Y_test[i]=np.array([1,1,1,1])#acms

In [ ]:
print(f"X_train:{X_train.shape}")
print(f"Y_train:{Y_train.shape}")
print(f"X_test:{X_test.shape}")
print(f"Y_test:{Y_test.shape}")

X_train:(34089, 46)
Y_train:(34089, 4)
X_test:(5662, 46)
Y_test:(5662, 4)


### 至此重采样完成，接下来开始进行分类模型构建

### 进行特征选择

In [ ]:
#使用卡方检验进行特征选择
X=np.concatenate((X_train,X_test),axis=0)
Y=np.concatenate((Y_train,Y_test),axis=0)
X_new=SelectPercentile(chi2,percentile=50).fit_transform(X,Y)
X_train=X_new[0:X_train.shape[0]]
X_test=X_new[X_train.shape[0]:]
print(X_train.shape)
print(X_test.shape)

In [ ]:
def get_knn(x,X_train,k,sign):
    diffMat=np.tile(x,(X_train.shape[0],1))-X_train
    distance=((diffMat**2).sum(axis=1))**0.5
    if sign==1:
        ans=distance.argsort()[0:k]
    else:
        ans=distance.argsort()[1:k+1]
    return ans

In [ ]:
#mlknn
def mlknn_train(X_train,Y_train,k):
    s=1
    #计算训练样本是第i类的概率
    p_1=np.zeros(4)
    p_0=np.zeros(4)
    for i in range(0,4):
        p_1[i]=(s+sum(Y_train[:,i]==1))/(s*2+Y_train.shape[0])
        p_0[i]=1-p_1[i]
    print(f"p_1:{p_1}")
    #计算某样本是第i类的条件下，它的K近邻中有几个是第i类的条件概率
    #c_1记录样本的K近邻中有count个属于这一类的情况下该样本也属于这一类的个数
    #c_0统计样本不属于这一类且其K近邻中有count个属于这一类的事件的个数
    c_1=np.zeros((4,k+1))
    c_0=np.zeros((4,k+1))
    #p_1_n记录样本属于第i类的情况下，其K近邻中有几个是第i类的条件概率
    p_1_n=np.zeros((4,k+1))
    p_0_n=np.zeros((4,k+1))
    N_x=np.zeros((X_train.shape[0],k,4))
    for j in range(0,X_train.shape[0]):
        N_x[j]=Y_train[get_knn(X_train[j],X_train,k,0)] #获得每个训练样本x的K近邻样本的标签
    for i in range(0,4):
        for j in range(0,Y_train.shape[0]):
            count=sum(N_x[j][:,i]==1)              #统计第j个训练样本的K近邻样本中属于第i类的数量
            if(Y_train[j][i]==1):
                c_1[i][count]+=1
            else:
                c_0[i][count]+=1
        #求出样本是第i类的条件下，K近邻中有j个是第i类的条件概率
        #求出样本不是第i类的条件下，K近邻中有j个是第i类的条件概率
        for j in range(0,k+1):
            p_1_n[i][j]=(s+c_1[i][j])/(s*(k+1)+sum(c_1[i]))
            p_0_n[i][j]=(s+c_0[i][j])/(s*(k+1)+sum(c_0[i]))
    return p_1,p_0,p_1_n,p_0_n

In [ ]:
def mlknn_predecit(X_test,X_train,Y_train,k,p_1,p_0,p_1_n,p_0_n):
    #获取待预测样本的K近邻样本的标签
    N_x=np.zeros((X_test.shape[0],k,4))
    Y_pre=np.zeros((X_test.shape[0],4))
    for j in range(0,X_test.shape[0]):
        N_x[j]=Y_train[get_knn(X_test[j],X_train,k,1)]
    for i in range(0,4):
        for j in range(0,X_test.shape[0]):
            count=sum(N_x[j][:,i]==1)              #统计第j个训练样本的K近邻样本中属于第i类的数量
            Y_pre[j][i]=(p_1[i]*p_1_n[i][count]>p_0[i]*p_0_n[i][count])
    return Y_pre

In [ ]:
def Accuracy(y_true, y_pred):#Hamming Score
    temp = 0
    for i in range(y_true.shape[0]):
        temp += sum(np.logical_and(y_true[i], y_pred[i])) / sum(np.logical_or(y_true[i], y_pred[i]))
    return temp / y_true.shape[0]

In [ ]:
def Absolute_True(y_true,y_pred):
    count=0
    for i in range(0,y_pred.shape[0]):
        if((y_pred[i]==y_true[i]).all()):
            count+=1
    return count/y_true.shape[0]

In [ ]:
#使用训练集进行K折交叉验证并返回相关评价指标
def mlknn_KfoldCV(X,Y,k,kfold):
    Aiming=0;Coverage=0;Acc=0;A_T=0;A_F=0
    kf=KFold(n_splits=kfold,shuffle=True)
    for train_index,test_index in kf.split(X):
        x_train,x_test=X[train_index],X[test_index]
        y_train,y_test=Y[train_index],Y[test_index]
        p_1,p_0,p_1_n,p_0_n=mlknn_train(x_train,y_train,k)
        y_pred=mlknn_predecit(x_test,x_train,y_train,k,p_1,p_0,p_1_n,p_0_n)
        Aiming+=precision_score(y_true=y_test,y_pred=y_pred,average='samples',zero_division=1)#平均精确率
        Coverage+=recall_score(y_true=y_test,y_pred=y_pred,average='samples')
        Acc+=Accuracy(y_test,y_pred)
        A_T+=Absolute_True(y_test,y_pred)
        A_F+=hamming_loss(y_test,y_pred)
    return Aiming/kfold,Coverage/kfold,Acc/kfold,A_T/kfold,A_F/kfold

In [ ]:
#使用测试集进行测试并返回相关评价指标
def mlknn_test(X_test,Y_test,X_train,Y_train,k):
    p_1,p_0,p_1_n,p_0_n=mlknn_train(X_train,Y_train,k)
    print("finish training,start predict")
    y_pred=mlknn_predecit(X_test,X_train,Y_train,k,p_1,p_0,p_1_n,p_0_n)
    Aiming=0;Coverage=0;Acc=0;A_T=0;A_F=0
    Aiming=precision_score(y_true=Y_test,y_pred=y_pred,average='samples',zero_division=1)#平均精确率
    Coverage=recall_score(y_true=Y_test,y_pred=y_pred,average='samples')
    Acc=Accuracy(Y_test,y_pred)
    A_T=Absolute_True(Y_test,y_pred)
    A_F=hamming_loss(Y_test,y_pred)
    return Aiming,Coverage,Acc,A_T,A_F

In [ ]:
Aiming_t,Coverage_t,Acc_t,A_T_t,A_F_t=mlknn_test(X_test,Y_test,X_train,Y_train,4)
print(f"Aiming:{Aiming_t}")
print(f"Coverage:{Coverage_t}")
print(f"Acc:{Acc_t}")
print(f"A_T:{A_T_t}")
print(f"A_F:{A_F_t}")

p_1:[0.63635564 0.54545188 0.45454812 0.36364436]
finish training,start predict
Aiming:0.8081213940892499
Coverage:0.8038090191922759
Acc:0.7614506063817335
A_T:0.6813846697280113
A_F:0.14698869657364888


In [ ]:
Aiming,Coverage,Acc,A_T,A_F=mlknn_KfoldCV(X_train,Y_train,4,5)
print(f"Aiming:{Aiming}")
print(f"Coverage:{Coverage}")
print(f"Acc:{Acc}")
print(f"A_T:{A_T}")
print(f"A_F:{A_F}")

p_1:[0.633667   0.54585121 0.45165548 0.36636967]
p_1:[0.63777362 0.54621787 0.45510212 0.36325303]
p_1:[0.63729696 0.54742786 0.4532688  0.36482969]
p_1:[0.63729696 0.54299124 0.45447879 0.36288637]
p_1:[0.63573367 0.54476791 0.45823862 0.36089316]
Aiming:0.9769427016839497
Coverage:0.973671867472967
Acc:0.9683915588489823
A_T:0.959840395215781
A_F:0.0158628984885981
